# Construct Fine-Tuned SST-2 Model Using AdapterHub

Install AdapterHub

In [1]:
!pip install adapter-transformers

     |████████████████████████████████| 683 kB 2.8 MB/s 
     |████████████████████████████████| 5.6 MB 7.3 MB/s 
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.8.1rc1
    Uninstalling tokenizers-0.8.1rc1:
      Successfully uninstalled tokenizers-0.8.1rc1
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

transformers 3.0.2 requires tokenizers==0.8.1.rc1, but you'll have tokenizers 0.7.0 which is incompatible.


Freeze requirements for later reference.

In [2]:
!pip freeze > kaggle_image_requirements.txt

Load required tranformers classes

In [3]:
from transformers import BertForSequenceClassification, BertTokenizer

wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.


Load adapters for the `bert-base-uncased` checkpoint, task SST-2 as fine-tuned on the UKP Sentential Argument Mining Corpus - https://www.informatik.tu-darmstadt.de/ukp/research_6/data/argumentation_mining_1/ukp_sentential_argument_mining_corpus/index.en.jsp

Available adapters for SST-2 in AdapterHub, and usage instructions, can be found here - https://adapterhub.ml/explore/sentiment/sst-2/. 

General adapter repository is naturally https://adapterhub.ml/explore/

In [4]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")
model.load_adapter("sentiment/sst-2@ukp")

'sst-2'

That is all! You have loaded the fine-tuned model using an adapter someone else shared! We can use the fine-tuned model as follows to make a prediction about the sentiment of a given sentence

In [5]:
import torch

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased") # Use regular pretrained tokenizer
tokensA = tokenizer.tokenize("That was an amazing contribution, good!") # Sentence A
input_tensorA = torch.tensor([tokenizer.convert_tokens_to_ids(tokensA)])
tokensB = tokenizer.tokenize("That is bad for the environment.") # Sentence B
input_tensorB = torch.tensor([tokenizer.convert_tokens_to_ids(tokensB)])
outputsA = model(input_tensorA,adapter_names=['sst-2']) # make prediction A
outputsB = model(input_tensorB,adapter_names=['sst-2']) # make prediction B

Display prediction outputs. The output should be interpreted as [negative sentiment probability, positive sentiment probability]

In [6]:
print("The prediction for sentence A - That was an amazing contribution, good! - is:")
print(torch.nn.functional.softmax(outputsA[0][0])) # display prediction probabilities
print("The prediction for sentence B - That is very bad for the environment. - is:")
print(torch.nn.functional.softmax(outputsB[0][0]))  # display prediction probabilities

The prediction for sentence A - That was an amazing contribution, good! - is:
tensor([0.0010, 0.9990], grad_fn=<SoftmaxBackward>)
The prediction for sentence B - That is very bad for the environment. - is:
tensor([0.8156, 0.1844], grad_fn=<SoftmaxBackward>)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  after removing the cwd from sys.path.
